In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import scipy.signal
import scipy.special
from scipy.special import erfc
from math import log, log2, sqrt
import ipywidgets as widgets
from ipywidgets import IntSlider, FloatSlider, interactive, Layout, Dropdown, IntText, HBox, VBox, Output
from IPython.display import display, clear_output
import time
print("Libraries added successfully!")

Libraries added successfully!


In [2]:
# The function grayCode accepts a number n and returns an array g containing
# the Gray coding for numbers with n bits.
# This is done by recursively calling the function gray_code_recurse
def grayCode(n):
    def gray_code_recurse (g,n):
        k=len(g)
        if n<=0:
            return
        else:
            for i in range (k-1,-1,-1):
                char='1'+g[i]
                g.append(char)
            for i in range (k-1,-1,-1):
                g[i]='0'+g[i]

            gray_code_recurse (g,n-1)

    g=['0','1']
    gray_code_recurse(g,n-1)
    return g


# The function naturalBinaryCoding accepts a number n and returns an array
# with binary numbers with n bits (or equivalently up to the number 2**n, which results
# by left shifting 1 by n bits)
def naturalBinaryCoding(n):
    binary_levels = []
    for i in range(1 << n):
        binary_levels.append('{:0{}b}'.format(i, n))
    return binary_levels


# The function generateRandomBits generates n_bits random binary digits
def generateRandomBits(n_bits):
    bitstream = []
    for i in range(n_bits):
        random_bit = random.randint(0, 1)
        bitstream.append(random_bit)
    return bitstream


# The function createLevels divides the range [-A, A] into L-1 equal intervals,
# so that it always contains the endpoints of the range
def createLevels(A, L):
    y = []
    step = (A - (-A)) / (L - 1)
    for i in range(L):
        y.append(-A + i * step)
    return y


# The function createSymbols takes an array with bits as an argument and
# groups neighboring digits into symbols with length k
def createSymbols(k, bitstream):
    n_bits = len(bitstream)
    symbols = []
    for i in range(0, n_bits - k + 1, k):
        symbol = ""
        for j in range(k):
            symbol += str(bitstream[i+j])
        symbols.append(symbol)
    return symbols


# The function rootRaisedCosine creates a root raised cosine pulse
# with a roll-off coefficient and order determined by the sampling rate (nsample)
# and the delay it will introduce (delay)
def rootRaisedCosine(nsamp, roll_off, delay):
    F0 = 0.5 / nsamp
    Fd = 1
    Fs = Fd * nsamp
    Td = 1 / Fd
    Ts = 1 / Fs
    F1 = F0 * (1 - roll_off)
    F2 = F0 * (1 + roll_off)
    filter_order = 2 * nsamp * delay

    t = np.arange(0, filter_order, Td)
    h = []
    for i in range(len(t)):
        t_shifted = t[i] - filter_order / 2
        if t_shifted == 0:
            h.append(np.sqrt(2 * F0) *(1 + roll_off * ((4 / np.pi) - 1)))
        elif t_shifted == 1 / 8 / roll_off / F0 or t_shifted == - 1 / 8 / roll_off / F0 :
            h.append((roll_off * np.sqrt(F0)) * ((1 + 2 / np.pi) * np.sin(np.pi / 4 / roll_off) + (1 - 2 / np.pi) * np.cos(np.pi / 4 / roll_off)))
        else:
            factor1 = np.sqrt(2 * F0) / (1 - 64 * roll_off* roll_off * F0 * F0 * t_shifted * t_shifted)
            factor2 = np.sin(2 * np.pi * F1 * t_shifted) / (2 * np.pi * F0 * t_shifted)
            factor3 = (4 * roll_off / np.pi) * np.cos(2 * np.pi * F2 * t_shifted) 
            h.append(factor1 * (factor2 + factor3))
    
    return t,h


# The function upSample increases the number of samples of a signal signal by adding nsamp-1
# zeros after each sample of the signal
def upSample(signal, nsamp):
    upSampled = []
    for i in range(len(signal) * nsamp):
        if i % nsamp == 0:
            upSampled.append(signal[i // nsamp])
        else:
            upSampled.append(0)
    return upSampled


# The function downSample reduces the sampling frequency of a signal signal by a factor of nsamp
# by keeping only the samples that are multiples of nsamp (0, nsamp, 2*nsamp, ...)
def downSample(signal, nsamp):
    downSampled = []
    for i in range(0, len(signal), nsamp):
        downSampled.append(signal[i])
        
    return downSampled


# Adds white Gaussian noise with mean value μ (mu) and variance σ^2 (sigma)
def generateAWGN(signal, mu, sigma):
    noise = sigma * np.random.randn(len(signal)) + mu
    return noise


# Loading animation
loading = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div class='loader' style='border: 12px solid #f3f3f3; /* Light grey */
                                     border-top: 12px solid #01cc97; /* Blue */
                                     border-radius: 50%;
                                     width: 40px;
                                     height: 40px;
                                     animation: spin 2s linear infinite;'></div>
    </div>
    <style>
    @keyframes spin {
        0% { transform: rotate(0deg); }
        100% { transform: rotate(360deg); }
    }
    </style>
    """
done = """
        <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
            <div style='font-size: 40px; color: #01cc97;'>&#10003;</div>
        </div>
        """
loader_html1 = widgets.HTML(
  value=loading
)
timer_html1 = widgets.HTML(
    value="Elapsed time: - seconds"
)

# Output widget for dynamic updates
output = Output()

global filt, g_delay, g_nsamp
def interactive_signal_processing1(n_bits, L, roll_off, nsamp, delay):
    # Start timer
    loader_html1.value = loading
    start_time = time.time()

    global filt, g_delay, g_nsamp
    g_delay = delay
    g_nsamp = nsamp
    bitstream = generateRandomBits(n_bits)
    A = L - 1
    k = int(log2(L))
    y_levels = createLevels(A, L)
    symbols = createSymbols(k, bitstream)
    gray_encoding = grayCode(k)

    x_gray = [y_levels[gray_encoding.index(symbol)] for symbol in symbols]
    t_filter, filt = rootRaisedCosine(nsamp, roll_off, delay)
    y = upSample(x_gray, nsamp)
    y_transmitted = scipy.signal.convolve(y, filt)
    y_received = scipy.signal.convolve(y_transmitted, filt)
    y_final = downSample(y_received, nsamp)
    y_final = y_final[2 * delay: len(y_final) - 2 * delay]
    with output:
        output.clear_output(wait=True)  # Clear previous output
        fig = plt.figure(figsize=(20, 12))  # Create a figure object

        # First subplot - Root Raised Cosine Filter
        ax1 = fig.add_subplot(2, 2, 1)  # First plot in the first row, first column
        ax1.plot(t_filter, filt)
        ax1.set_title('Root Raised Cosine Filter')
        ax1.set_xlabel('Time')
        ax1.set_ylabel('Amplitude')
        ax1.grid(True)

        # Second subplot - Continuous signal with stems
        ax2 = fig.add_subplot(2, 2, 2)  # Second plot in the first row, second column
        t_continuous = np.arange(0, len(y_transmitted[:10*nsamp]))  # Time vector for continuous signal
        ax2.plot(t_continuous, y_transmitted[:10*nsamp], label='Filtered Signal')

        # Generating stem positions for original symbols
        t_symbols = np.arange(0, 10*nsamp, nsamp)  # Stem positions every nsamp samples

        # Get the corresponding y-values from y_transmitted at the stem positions
        y_stems = y_transmitted[t_symbols]

        ax2.stem(t_symbols, y_stems, linefmt='C1-', markerfmt='C1o', basefmt=" ", label='Original Symbols')

        ax2.set_title('Signal Visualization')
        ax2.set_xlabel('Time')
        ax2.set_ylabel('Amplitude')
        ax2.legend()
        ax2.grid(True)

        # Third subplot - Signal Visualization
        ax3 = fig.add_subplot(2, 2, 3)  # Third plot in the second row, first column
        t = np.arange(0, len(y[:10]))
        ax3.plot(t, y_final[:10])
        ax3.stem(t, x_gray[:10])
        ax3.set_title('Signal Visualization')
        ax3.legend(['Received','Transmittted'])
        ax3.set_xlabel('Time')
        ax3.set_ylabel('Amplitude')
        ax3.grid(True)

        # Fourth subplot - Power Spectral Density of the Received Signal
        ax4 = fig.add_subplot(2, 2, 4)  # Fourth plot in the second row, second column
        f, Pxx_den = scipy.signal.welch(y_received, window='hamming', nperseg=8192)
        Pxx_den = 10 * np.log10(Pxx_den)
        ax4.plot(f, Pxx_den)
        ax4.set_title('Power Spectral Density of the Received Signal')
        ax4.set_xlabel('Normalized Frequency')
        ax4.set_ylabel('Power/Frequency [dB]')
        ax4.grid(True)

        plt.tight_layout()  # Adjust the layout
            
        # Show elapsed time
        elapsed_time = time.time() - start_time
        timer_html1.value = f"Elapsed time: {elapsed_time:.2f} seconds"
        loader_html1.value = done
        
        plt.show()


# Widget setup
n_bits_slider1 = IntSlider(min=10000, max=100000, step=10000, value=10000, description='Bitstream Length', style={'description_width': 'initial'}, layout=Layout(width='100%'), continuous_update=False)
roll_off_slider1 = FloatSlider(min=0.1, max=1.0, step=0.1, value=0.4, description='Roll-off Factor', style={'description_width': 'initial'}, layout=Layout(width='100%'), continuous_update=False)
L_dropdown1 = Dropdown(options=[2**i for i in range(1, 6)], value=2, description='ASK Levels', style={'description_width': 'initial'}, continuous_update=False)
nsamp_slider1 = IntSlider(min=10, max=40, step=1, value=20, description='nsamp', style={'description_width': 'initial'}, layout=Layout(width='100%'), continuous_update=False)
delay_slider1 = IntSlider(min=1, max=10, step=1, value=5, description='Group Delay', style={'description_width': 'initial'}, layout=Layout(width='100%'), continuous_update=False)

# Create the interactive output for the interactive_signal_processing1 function
out = widgets.interactive_output(interactive_signal_processing1, 
                                 {'n_bits': n_bits_slider1, 
                                  'L': L_dropdown1, 
                                  'roll_off': roll_off_slider1, 
                                  'nsamp': nsamp_slider1, 
                                  'delay': delay_slider1})

# Update filter order when sliders change
def update_filter_order1(*args):
    filter_order1 = 2 * nsamp_slider1.value * delay_slider1.value
    filter_order_display1.value = filter_order1

# Observers to trigger filter order update on slider value changes
nsamp_slider1.observe(update_filter_order1, 'value')
delay_slider1.observe(update_filter_order1, 'value')

# Display the filter order with an IntText widget
filter_order_display1 = widgets.IntText(value=2 * nsamp_slider1.value * delay_slider1.value, 
                                        description='Filter Order:', 
                                        disabled=True)

# Arrange the input widgets in a VBox layout
inputs = widgets.VBox([n_bits_slider1, roll_off_slider1, nsamp_slider1, delay_slider1, filter_order_display1, L_dropdown1])

# Group the loader and timer together (they will appear next to each other horizontally)
loader_timer_box = widgets.VBox([loader_html1, timer_html1], layout=widgets.Layout(margin='0 0 0 20px', width='auto'))

# Arrange the UI with HBox and layout options
ui = widgets.HBox([inputs, loader_timer_box], layout=widgets.Layout(align_items='center', flex_flow='row nowrap'))
inputs.layout.flex = '1 1 auto'  # Flex-grow, flex-shrink, flex-basis for input widgets
loader_timer_box.layout.flex = '0 1 auto'  # No flex-grow for loader/timer box

# Display the UI components and the interactive output
display(ui, out)

# Initial update of filter order
update_filter_order1()

Output()

In [3]:
# Loading animation
loading = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div class='loader' style='border: 12px solid #f3f3f3; /* Light grey */
                                     border-top: 12px solid #01cc97; /* Blue */
                                     border-radius: 50%;
                                     width: 40px;
                                     height: 40px;
                                     animation: spin 2s linear infinite;'></div>
    </div>
    <style>
    @keyframes spin {
        0% { transform: rotate(0deg); }
        100% { transform: rotate(360deg); }
    }
    </style>
    """
done = """
        <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
            <div style='font-size: 40px; color: #01cc97;'>&#10003;</div>
        </div>
        """
loader_html2 = widgets.HTML(
  value=loading
)
timer_html2 = widgets.HTML(
    value="Elapsed time: - seconds"
)

# This function wraps the signal processing and BER calculation, making parameters adjustable via sliders
def interactive_signal_processing2(n_bits=80000, roll_off=0.7, nsamp=16, delay=4, L=16):
    # Start timer
    loader_html2.value = loading
    start_time = time.time()

    bitstream = generateRandomBits(n_bits)
    A = L - 1
    k = int(log2(L))
    y_levels = createLevels(A, L)
    symbols = createSymbols(k, bitstream)
    gray_encoding = grayCode(k)

    x_gray = [y_levels[gray_encoding.index(symbol)] for symbol in symbols]
    t_filter, filt = rootRaisedCosine(nsamp, roll_off, delay)
    y = upSample(x_gray, nsamp)
    y_transmitted = scipy.signal.convolve(y, filt)

    # Calculating BER theoretically and via simulation
    berTheoretical = []
    berSimulation = []
    EbN0_max = 20

    for EbN0_db in range(1, EbN0_max):
        EbN0 = 10 ** (EbN0_db / 10)
        SNR_db = EbN0_db - 10 * np.log10(nsamp / 2 / k)
        SNR = 10 ** (SNR_db / 10)
        P = sum(y_transmitted * y_transmitted) / len(y_transmitted)
        Pn = P / SNR

        noise = np.random.normal(0, sqrt(Pn), len(y_transmitted))
        y_noisy = y_transmitted + noise

        z_received = scipy.signal.convolve(y_noisy, filt)
        z_final = downSample(z_received, nsamp)
        z_final = z_final[2 * delay : len(z_final) - 2 * delay]

        errors = 0
        for i in range(len(z_final)):
            differences = np.abs(y_levels - z_final[i])
            index = np.argmin(differences)
            if y_levels[index] != x_gray[i]:
                errors += 1
        
        berSimulation.append(errors / n_bits)
        Pe = (L - 1) / L * erfc(sqrt(3 * k / (L**2 - 1) * EbN0))
        berTheoretical.append(Pe / k)
    
    # Plotting the BER
    plt.figure(figsize=(10, 6))
    plt.semilogy(berTheoretical, label='Theoretical')
    plt.semilogy(berSimulation, 'o', label='Simulation')
    plt.xlabel('Eb/N0 (dB)')
    plt.ylabel('Bit Error Rate')
    plt.title('BER Curve for L-ASK')
    plt.legend()
    plt.grid(True)
    
    # Show elapsed time
    elapsed_time = time.time() - start_time
    timer_html2.value = f"Elapsed time: {elapsed_time:.2f} seconds"
    loader_html2.value = done

    plt.show()

# Creating sliders and dropdown
n_bits_slider2 = IntSlider(min=10000, max=100000, step=10000, value=80000, description='Binary Sequence Length', style={'description_width': 'initial'}, layout=Layout(width='100%'), continuous_update=False)
roll_off_slider2 = FloatSlider(min=0.1, max=1.0, step=0.1, value=0.7, description='Roll-off Factor', style={'description_width': 'initial'}, layout=Layout(width='100%'), continuous_update=False)
nsamp_slider2 = IntSlider(min=8, max=32, step=1, value=16, description='nsamp', style={'description_width': 'initial'}, layout=Layout(width='100%'), continuous_update=False)
delay_slider2 = IntSlider(min=1, max=8, step=1, value=4, description='Group Delay', style={'description_width': 'initial'}, layout=Layout(width='100%'), continuous_update=False)
L_dropdown2 = Dropdown(options=[2**i for i in range(1, 6)], value=16, description='ASK Levels', style={'description_width': 'initial'}, continuous_update=False)

# Create the interactive output for the interactive_signal_processing2 function
out = widgets.interactive_output(interactive_signal_processing2, 
                                 {'n_bits': n_bits_slider2, 
                                  'roll_off': roll_off_slider2, 
                                  'nsamp': nsamp_slider2, 
                                  'delay': delay_slider2, 
                                  'L': L_dropdown2})

# Display filter order
filter_order_display2 = widgets.IntText(value=2 * nsamp_slider2.value * delay_slider2.value, 
                                        description='Filter Order:', 
                                        disabled=True)

# Define the update function to change the filter order dynamically
def update_filter_order2(*args):
    filter_order2 = 2 * nsamp_slider2.value * delay_slider2.value
    filter_order_display2.value = filter_order2

# Attach observers to sliders to update filter order when values change
nsamp_slider2.observe(update_filter_order2, 'value')
delay_slider2.observe(update_filter_order2, 'value')

# Group the input widgets into a VBox
input_widgets = widgets.VBox([n_bits_slider2, roll_off_slider2, nsamp_slider2, delay_slider2, 
                              filter_order_display2, L_dropdown2], 
                             layout=widgets.Layout(flex='1 1 auto', width='auto'))

# Group the loader and timer together (they will appear next to each other horizontally)
loader_timer_box = widgets.VBox([loader_html2, timer_html2], layout=widgets.Layout(margin='0 0 0 20px', width='auto'))

# Combine input widgets and loader/timer into a horizontal layout (HBox)
ui = widgets.HBox([input_widgets, loader_timer_box], layout=widgets.Layout(align_items='center'))

# Display the UI components and the interactive output
clear_output(wait=True)  # Clear the previous output
display(ui, out)

Output()

In [4]:
# Loading animation
loading = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div class='loader' style='border: 12px solid #f3f3f3; /* Light grey */
                                     border-top: 12px solid #01cc97; /* Blue */
                                     border-radius: 50%;
                                     width: 40px;
                                     height: 40px;
                                     animation: spin 2s linear infinite;'></div>
    </div>
    <style>
    @keyframes spin {
        0% { transform: rotate(0deg); }
        100% { transform: rotate(360deg); }
    }
    </style>
    """
done = """
        <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
            <div style='font-size: 40px; color: #01cc97;'>&#10003;</div>
        </div>
        """
loader_html3 = widgets.HTML(
  value=loading
)
timer_html3 = widgets.HTML(
    value="Elapsed time: - seconds"
)

def interactive_signal_processing_with_natural(n_bits=80000, roll_off=0.7, nsamp=16, delay=4, L=16):
    # Start timer
    loader_html3.value = loading
    start_time = time.time()

    # Common setup
    bitstream = generateRandomBits(n_bits)
    A = L - 1
    k = int(log2(L))
    y_levels = createLevels(A, L)
    symbols = createSymbols(k, bitstream)

    # Gray encoding setup
    gray_encoding = grayCode(k)
    x_gray = [y_levels[gray_encoding.index(symbol)] for symbol in symbols]

    # Natural encoding setup
    natural_encoding = naturalBinaryCoding(k)
    x_natural = [y_levels[natural_encoding.index(symbol)] for symbol in symbols]

    # Root raised cosine filter (common for both Gray and Natural)
    t_filter, filt = rootRaisedCosine(nsamp, roll_off, delay)

    # Transmission simulation for Gray encoding
    y_gray = upSample(x_gray, nsamp)
    y_transmitted_gray = scipy.signal.convolve(y_gray, filt)

    # Transmission simulation for Natural encoding
    y_natural = upSample(x_natural, nsamp)
    y_transmitted_natural = scipy.signal.convolve(y_natural, filt)

    # Calculate BER for Gray encoding
    berTheoretical = []
    berSimulation_gray = []
    berSimulation_natural = [] # For natural encoding
    EbN0_max = 20

    for EbN0_db in range(1, EbN0_max):
        # Convert Eb/N0 from dB to linear scale
        EbN0 = 10 ** (EbN0_db * 0.1)
        SNR_db = EbN0_db - 10*np.log10(nsamp/2/k)
        SNR = 10 ** (SNR_db * 0.1)

        # Calculate the power of the transmitted signal and the noise power to achieve the desired SNR
        P = sum(y_transmitted_gray * y_transmitted_gray) / len(y_transmitted_gray)
        P_db = 10 * np.log10(P)
        Pn_db = P_db - SNR_db
        Pn = 10 ** (Pn_db * 0.1)

        # Add noise (common for both Gray and Natural)
        mu = 0
        sigma = np.sqrt(Pn)
        noise = generateAWGN(y_transmitted_gray, mu, sigma)
        y_noisy_gray = y_transmitted_gray + noise
        y_noisy_natural = y_transmitted_natural + noise

        # Receiver simulation for Gray encoding
        z_received_gray = scipy.signal.convolve(y_noisy_gray, filt)
        z_final_gray = downSample(z_received_gray, nsamp)
        z_final_gray = z_final_gray[2 * delay : len(z_final_gray) - 2 * delay]

        # Receiver simulation for Natural encoding (Code 1)
        z_received_natural = scipy.signal.convolve(y_noisy_natural, filt)
        z_final_natural = downSample(z_received_natural, nsamp)
        z_final_natural = z_final_natural[2 * delay : len(z_final_natural) - 2 * delay]

        # Error calculation for Gray encoding
        # Decision for the level corresponding to the symbol received for Gray encoding
        for i in range(len(z_final_gray)):
            # Array with the differences of the signal from the levels
            differences = np.abs(y_levels - z_final_gray[i]) 
            m = min(differences) # Find the minimum distance
            [index], = np.where(differences == m)
            z_final_gray[i] = y_levels[index]
        
        # We assume that each error in Gray encoding results in one incorrect bit
        error = 0
        for i in range(len(z_final_gray)):
            if x_gray[i] != z_final_gray[i]:
                error += 1
        
        # BER calculation = number of errors / number of bits
        berSimulation_gray.append(error/n_bits)

        # Error calculation for Natural encoding 
        # Decision for the level corresponding to the symbol received
        for i in range(len(z_final_natural)):
            differences = np.abs(y_levels - z_final_natural[i])
            m = min(differences)
            [index], = np.where(differences == m)
            z_final_natural[i] = y_levels[index]
        
        # The final_symbols contains the bits that were decided to be received
        final_symbols = []
        for i in range(len(z_final_natural)):
            # Mapping each decided level to its binary encoding
            index = y_levels.index(z_final_natural[i])
            final_symbols.append(natural_encoding[index])

        # For each incorrect symbol, we need to check how many bits were wrong,
        # as the encoding of neighboring levels no longer differs by only one bit 
        error = 0
        for i in range(len(z_final_natural)):
            # If a symbol is wrong, check how many digits were received incorrectly
            if x_natural[i] != z_final_natural[i]:
                for j in range(len(symbols[i])):
                    if symbols[i][j] != final_symbols[i][j]:
                        error += 1

        berSimulation_natural.append(error/n_bits)
        # Theoretical BER calculation (common for both Gray and Natural)
        Pe = (L - 1) / L * erfc(sqrt(3 * k / (L**2 - 1) * EbN0))
        berTheoretical.append(Pe / k)

    # Plotting the BER for both Gray and Natural encoding
    plt.figure(figsize=(10, 6))
    plt.semilogy(range(1, EbN0_max), berTheoretical, label='Theoretical')
    plt.semilogy(range(1, EbN0_max), berSimulation_gray, 'o', label='Gray')
    plt.semilogy(range(1, EbN0_max), berSimulation_natural, '*', label='Natural')
    plt.xlabel('Eb/N0 (dB)')
    plt.ylabel('Bit Error Rate')
    plt.title('BER Curve for L-ASK')
    plt.legend()
    plt.grid(True)

    # Show elapsed time
    elapsed_time = time.time() - start_time
    timer_html3.value = f"Elapsed time: {elapsed_time:.2f} seconds"
    loader_html3.value = done

    plt.show()


# Creating sliders and dropdown
n_bits_slider1 = IntSlider(min=10000, max=100000, step=10000, value=80000, description='Binary Sequence Length', style={'description_width': 'initial'}, layout=Layout(width='100%'), continuous_update=False)
roll_off_slider1 = FloatSlider(min=0.1, max=1.0, step=0.1, value=0.7, description='Roll-off Factor', style={'description_width': 'initial'}, layout=Layout(width='100%'), continuous_update=False)
nsamp_slider1 = IntSlider(min=8, max=32, step=1, value=16, description='nsamp', style={'description_width': 'initial'}, layout=Layout(width='100%'), continuous_update=False)
delay_slider1 = IntSlider(min=1, max=8, step=1, value=4, description='Group Delay', style={'description_width': 'initial'}, layout=Layout(width='100%'), continuous_update=False)
L_dropdown1 = Dropdown(options=[2**i for i in range(1, 6)], value=16, description='ASK Levels', style={'description_width': 'initial'}, continuous_update=False)

# Reordering the widgets
interactive_plot_with_natural = interactive(interactive_signal_processing_with_natural, n_bits=n_bits_slider1, roll_off=roll_off_slider1, nsamp=nsamp_slider1, delay=delay_slider1, L=L_dropdown1)

input_widgets = VBox([n_bits_slider1, roll_off_slider1, nsamp_slider1, delay_slider1, L_dropdown1], layout=Layout(flex='1 1 auto', width='auto'))
plot_output = interactive_plot_with_natural.children[-1]  # The output plot

# Group the loader and timer together (they will appear next to each other horizontally)
loader_timer_box = widgets.VBox([loader_html3, timer_html3], layout=widgets.Layout(margin='0 0 0 20px', width='auto'))

# Create a VBox that includes both the input widgets and the loading animation
inputs_and_loader = HBox([input_widgets, loader_timer_box])

# Create an HBox to hold everything in a horizontal layout
ui = VBox([inputs_and_loader, plot_output])

# Display the UI components
clear_output(wait=True)  # Clear the previous output
display(ui)